In [36]:
# !pip install networkx
# !pip install pymorphy2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    NamesExtractor,
    Doc
)
from navec import Navec
import networkx as nx
from spacy_conll import init_parser
from spacy_conll.parser import ConllParser

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

In [5]:
# !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
# emb = Navec.load("navec_hudlit_v1_12B_500K_300d_100q.tar")
emb = NewsEmbedding()

In [6]:
# !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
# tagger = NERTagger(emb, )

In [7]:
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [1]:
text = """В этом же домике жил известный доктор Пилюлькин, который лечил коротышек от всех болезней. Он всегда ходил в белом халате, а на голове носил белый колпак с кисточкой. Жил здесь также знаменитый механик Винтик со своим помощником Шпунтиком; жил Сахарин Сахариныч Сиропчик, который прославился тем, что очень любил газированную воду с сиропом. Он был очень вежливый. Ему нравилось, когда его называли по имени и отчеству, и не нравилось, когда кто-нибудь называл его просто Сиропчиком. Жил еще в этом доме охотник Пулька. У него была маленькая собачка Булька и еще было ружье, которое стреляло пробками. Жил художник Тюбик, музыкант Гусля и другие малыши: Торопыжка, Ворчун, Молчун, Пончик, Растеряйка, два брата -- Авоська и Небоська. Но самым известным среди них был малыш, по имени Незнайка. Его прозвали Незнайкой за то, что он ничего не знал. Этот Незнайка носил яркую голубую шляпу, желтые, канареечные, брюки и оранжевую рубашку с зеленым галстуком. Он вообще любил яркие краски. Нарядившись таким попугаем, Незнайка по целым дням слонялся по городу, сочинял разные небылицы и всем рассказывал. Кроме того, он постоянно обижал малышек. Поэтому малышки, завидев издали его оранжевую рубашку, сейчас же поворачивали в обратную сторону и прятались по домам. У Незнайки был друг, по имени Гунька, который жил на улице Маргариток. С Гунькой Незнайка мог болтать по целым часам. Они двадцать раз на день ссорились между собой и двадцать раз на день мирились."""


In [8]:
doc = Doc(text)

In [9]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for token in doc.tokens:
    token.lemmatize(morph_vocab)
    
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [10]:
doc.sents[0].morph.print()

                   В ADP
                этом DET|Case=Loc|Gender=Masc|Number=Sing
                  же PART
              домике NOUN|Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing
                 жил VERB|Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
           известный ADJ|Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
              доктор NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
           Пилюлькин PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
                   , PUNCT
             который PRON|Case=Nom|Gender=Masc|Number=Sing
               лечил VERB|Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
           коротышек NOUN|Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing
                  от ADP
                всех DET|Case=Gen|Number=Plur
            болезней NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur
                   . PUNCT


In [16]:
doc.sents[2].syntax.print()

                 Жил          
                 здесь        
                 также        
              ┌► знаменитый   amod
  ┌►┌───────┌─└─ механик      nsubj
┌─│ │     ┌─└──► Винтик       appos
│ │ │     │ ┌──► со           case
│ │ │     │ │ ┌► своим        det
│ │ │     └►└─└─ помощником   nmod
│ │ │       └──► Шпунтиком    appos
│ │ │         ┌► ;            punct
│ │ │ ┌────►┌─└─ жил          conj
│ │ │ │   ┌─└►┌─ Сахарин      xcomp
│ │ │ │   │   └► Сахариныч    flat:name
│ │ │ │   └────► Сиропчик     flat:name
│ │ │ │     ┌──► ,            punct
│ │ │ │     │ ┌► который      nsubj
│ │ └►│     └─└─ прославился  acl:relcl
│ │   │ ┌───└──► тем          obl
│ │   │ │ ┌────► ,            punct
│ │   │ │ │ ┌──► что          mark
│ │   │ │ │ │ ┌► очень        advmod
│ └───└─└►└─└─└─ любил        acl
│         │ └──► газированную obj
│         └────► воду         obj
│             ┌► с            case
└────────────►└─ сиропом      nmod
                 .            


In [31]:
s = doc.sents[0].syntax.tokens
# s

In [35]:
G = nx.DiGraph()

# for s in doc.sents:
#     syn = s.syntax
#     print(s.text)
#     print(syn.tokens)
tokens = doc.sents[0].syntax.tokens

from natasha.syntax import token_deps

for i in tokens:
    if i.rel not in ['case', 'punct']:
        G.add_node(i.id, text=i.text)
        G.add_edge(i.id, i.head_id, t=i.rel)

# nx.draw_planar(G, 
#         arrows=True, with_labels=True, font_weight='bold',
#         labels={i: G.nodes[i].get('text') for i in G.nodes}
# )
# plt.show()

# for nid in G.nodes:
#     node = s[nid]
#     print(s)/

# @dataclass
# class SentenceToken:
#     token_id: str
#     text: str
    

sent_graph = {i.id: (i.text, i.rel, []) for i in tokens}

for i in tokens:
    if i.head_id != '1_0':
        sent_graph[i.head_id][2].append(i.id)

print(sent_graph)

{'1_1': ('В', 'case', []), '1_2': ('этом', 'det', ['1_3']), '1_3': ('же', 'advmod', []), '1_4': ('домике', 'obl', ['1_1', '1_2']), '1_5': ('жил', 'root', ['1_4', '1_7', '1_16']), '1_6': ('известный', 'amod', []), '1_7': ('доктор', 'nsubj', ['1_6', '1_8', '1_11']), '1_8': ('Пилюлькин', 'appos', []), '1_9': (',', 'punct', []), '1_10': ('который', 'nsubj', []), '1_11': ('лечил', 'acl:relcl', ['1_9', '1_10', '1_12', '1_15']), '1_12': ('коротышек', 'obj', []), '1_13': ('от', 'case', []), '1_14': ('всех', 'det', []), '1_15': ('болезней', 'obl', ['1_13', '1_14']), '1_16': ('.', 'punct', [])}


In [37]:
doc.ner.print()

В этом же домике жил известный доктор Пилюлькин, который лечил 
                                      PER──────                
коротышек от всех болезней. Он всегда ходил в белом халате, а на 
голове носил белый колпак с кисточкой. Жил здесь также знаменитый 
механик Винтик со своим помощником Шпунтиком; жил Сахарин Сахариныч 
        PER───                     PER──────      PER───────────────
Сиропчик, который прославился тем, что очень любил газированную воду с
────────                                                              
 сиропом. Он был очень вежливый. Ему нравилось, когда его называли по 
имени и отчеству, и не нравилось, когда кто-нибудь называл его просто 
Сиропчиком. Жил еще в этом доме охотник Пулька. У него была маленькая 
PER───────                              PER───                        
собачка Булька и еще было ружье, которое стреляло пробками. Жил 
        PER───                                                  
художник Тюбик, музыкант Гусля и другие малыш

In [10]:
# !pip install deeppavlov
# !python -m deeppavlov install syntax_ru_syntagrus_bert
!pip install russian_tagsets

  Preparing metadata (setup.py) ... done
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24636 sha256=24c271c8c9fa26aa0c66586d743760d058ae52bcff808a1b09aa6b2888cc42a1
  Stored in directory: /home/petr/.cache/pip/wheels/5e/b4/26/9c17a7cdcfc6b8cf43111312f3e7c6abb8e583599e37c422f8
Successfully built russian-tagsets


In [12]:
from deeppavlov import build_model, configs
# model = build_model(configs.syntax.syntax_ru_syntagrus_bert, download=True)
model = build_model("ru_syntagrus_joint_parsing")#, download=True)
res = model(text.split('. '))
print(res)

2022-10-20 14:48:01.459 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ru_syntagrus_joint_parsing' as '/home/petr/.local/share/virtualenvs/scripts-SYm7cvvS/lib/python3.7/site-packages/deeppavlov/configs/syntax/ru_syntagrus_joint_parsing.json'
2022-10-20 14:48:02.97 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/petr/.deeppavlov/models/morpho_ru_syntagrus/tag.dict]
2022-10-20 14:48:23.17 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/petr/.deeppavlov/models/morpho_ru_syntagrus/model]


INFO:tensorflow:Restoring parameters from /home/petr/.deeppavlov/models/morpho_ru_syntagrus/model


2022-10-20 14:48:26.52 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!
2022-10-20 14:48:26.731 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/petr/.deeppavlov/models/syntax_ru_syntagrus/deps.dict]


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



2022-10-20 14:48:53.750 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/petr/.deeppavlov/models/syntax_ru_syntagrus/model_joint]


INFO:tensorflow:Restoring parameters from /home/petr/.deeppavlov/models/syntax_ru_syntagrus/model_joint
['1\tВ\tв\tADP\t_\t_\t4\tcase\t_\t_\n2\tэтом\tэтот\tDET\t_\tCase=Loc|Gender=Masc|Number=Sing\t4\tdet\t_\t_\n3\tже\tже\tPART\t_\t_\t2\tadvmod\t_\t_\n4\tдомике\tдомик\tNOUN\t_\tAnimacy=Inan|Case=Loc|Gender=Masc|Number=Sing\t5\tobl\t_\t_\n5\tжил\tжить\tVERB\t_\tAspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act\t0\troot\t_\t_\n6\tизвестный\tизвестный\tADJ\t_\tCase=Nom|Degree=Pos|Gender=Masc|Number=Sing\t7\tamod\t_\t_\n7\tдоктор\tдоктор\tNOUN\t_\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t5\tnsubj\t_\t_\n8\tПилюлькин\tпилюлькин\tPROPN\t_\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t7\tappos\t_\t_\n9\t,\t,\tPUNCT\t_\t_\t11\tpunct\t_\t_\n10\tкоторый\tкоторый\tPRON\t_\tCase=Nom\t11\tnsubj\t_\t_\n11\tлечил\tлечить\tVERB\t_\tAspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act\t7\tacl:relcl\t_\t_\n12\tкоротышек\tкоротышка\tNOUN\t_\tAn

In [16]:
# !pip install spacy spacy_conll
import spacy

/home/petr/.local/share/virtualenvs/scripts-SYm7cvvS/lib/python3.7/site-packages/spacy/displacy/__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)


ValueError: [E096] Invalid object passed to displaCy: Can only visualize `Doc` or Span objects, or dicts if set to `manual=True`.

In [18]:
# !python -m spacy download ru_core_news_sm

     |████████████████████████████████| 15.3 MB 1.1 MB/s            
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.8
    Uninstalling pymorphy2-0.8:
      Successfully uninstalled pymorphy2-0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 0.17.6 requires pydantic==1.3, but you have pydantic 1.10.2 which is incompatible.
deeppavlov 0.17.6 requires pymorphy2==0.8, but you have pymorphy2 0.9.1 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [1]:
import pickle
# from spacy_conll import init_parser
# from spacy_conll.parser import ConllParser

In [25]:
nlp = ConllParser(init_parser("ru_core_news_sm", "spacy"))

In [139]:
docs = [nlp.parse_conll_text_as_spacy(i) for i in res] 
# spacy.displacy.serve(doc)

In [140]:
pickle.dump(docs, open('spacy_text.dat', 'wb'))

In [2]:
docs = pickle.load(open('spacy_text.dat', 'rb'))

In [3]:
from spacy.symbols import nsubj, VERB, NOUN, appos, NAMES, punct, conj
from pprint import pprint
from dataclasses import dataclass

@dataclass
class Tkn:
    idx: int
    text: str
    dep: str
    pos: str
    child: list

    def __str__(self):
        return f"{self.text} [{self.dep}/{self.pos}]"

def generate(root):
    used = set()
    
    def _run(token):
        if token.idx in used:
            return None
        used.add(token.idx)

        if token.dep == conj:
            return None

        t = Tkn(token.idx, token.text, token.dep_, token.pos_, [])

        for ch in token.children:
            if ch.dep == punct: continue
            res = _run(ch)
            if res is not None:
                t.child.append(res)

        root = token.head
        if root.pos_ == 'VERB':
            res = _run(root)
            if res is not None:
                t.child.append(res)

        return t
    
    return _run(root)

for i in docs[0].sents:
    for token in i:
        if token.dep == nsubj:
            r = generate(token)

            def p(x: Tkn, lvl: int = 1):
                if x is None: return []
                lines = [str(x)]
                for i in x.child:
                    lines += [('  ' * lvl + l) for l in p(i, lvl+1)]
                return lines

            r = p(r)
            print(*r, sep="\n")
            print()
#     break


доктор [nsubj/NOUN]
  известный [amod/ADJ]
  Пилюлькин [appos/PROPN]
  лечил [acl:relcl/VERB]
      который [nsubj/PRON]
      коротышек [obj/NOUN]
      болезней [obl/NOUN]
            от [case/ADP]
            всех [det/DET]
  жил [ROOT/VERB]
      домике [obl/NOUN]
            В [case/ADP]
            этом [det/DET]
                    же [advmod/PART]

который [nsubj/PRON]
  лечил [acl:relcl/VERB]
      коротышек [obj/NOUN]
      болезней [obl/NOUN]
            от [case/ADP]
            всех [det/DET]

